In [7]:
import requests
import json
import os
import csv
import pandas as pd
import math
import numpy as np

MY_API_KEY=''

def get_chunk(key,start,count='25',date='2010-2020'):
    resp = requests.get(f"https://api.elsevier.com/content/search/scopus?query={key}&start={start}&count={count}&date={date}",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
    data=json.loads(resp.text.encode('utf-8'))
    return data

def get_queries(path_file):
    df=pd.read_csv(path_file)
    return df.to_dict(orient='records')

def write_result(path_file,fields_query,result,index,year):
    df=pd.read_csv(path_file)
    df.loc[index,f"{year}"]=math.trunc(int(result))
    # df[year]=np.floor(df.year).astype(int)
    df.to_csv(path_file,index=False,)

def compose_query(fields_query):
    query=f"{fields_query['filter1']}%28{fields_query['query']}%29+{fields_query['logic1']}+{fields_query['filter2']}%28"
    query=query+f"{fields_query['query']}%29+{fields_query['logic2']}+{fields_query['filter3']}%28{fields_query['query']}%29"
    return query
def main():
    queries=get_queries("query_iot.csv")
    _lista_query=[]
    for query in queries:
        result=query['result']
        query=compose_query(query)
        query={"query":query,"result":result}
        _lista_query.append(query)
    df=pd.DataFrame(_lista_query)
    df.to_csv("query_iot_presentazione",index=False)

    # for i,query in enumerate(queries):
    #             for year in range(2010,2021):
    #                 composed_query=compose_query(query)            
    #                 data=get_chunk(composed_query,start=0,count=25,date=year)    
    #                 total_results=int(data['search-results']['opensearch:totalResults'])            
    #                 write_result("query_iot_distriibuite.csv",query,total_results,i,year)

if __name__ == "__main__":
    main()